# Exploração das duas Bases de Histórico de Diesel VIBRA

## Histórico de Atualizações

<table>
    <thead>
        <tr>
            <th>Versão</th>
            <th>Data</th>
            <th>Descrição</th>
            <th>Autor</th>
            <th>Email</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>1.0</th>
            <td>26/04/2023</td>
            <td>
                <p>Exploração das duas bases de histórico de vendas da Vibra</p>
                1ª oritunda da volumetria - 1ª fase do projeto</br>
                2ª oriunda do novo processamento do hist diesel
            </td>
            <td>Marcel Quintela, , Sergio Urzedo Jr</td>
            <td>marcel.quintela@avanade.com, sergiourzedojr@gmail.com</td>
        </tr>       
    </tbody>
</table>

# Setup Inicial

Carregando linhas com configurações iniciais já escritas em notebook base

No bloco de comando "%run" não pode ter nenhum outra linha de comando ou comentário

In [1]:
%run nb00_Setup_Usinas{'blob_relative_path_raw': 'nuvem/Usinas/'}

# Leitura dos Dados 

In [2]:
usinas   = pd.read_parquet(abfss_path_enriched+'tancagens_impt_usinas.parquet',storage_options = {'linked_service' : linked_service_enriched})
 
arq_vol = 'abfss://general@stedlk01dtandev.dfs.core.windows.net/raw/mercado_potencial/volumetria/hist_diesel.csv'       # nova base de 28/04/23
arq_usi = abfss_path_raw + 'Vibra/hist_diesel.csv'

# ADD infos do Histórico de vendas da VIBRA

### Histórico fornacido pela Volumetria

In [3]:
col_names = pd.read_csv(arq_vol, nrows=0).columns                           # coletando informações de cabeçalho
col_types = {col: str for col in col_names if col not in {}}                # todas as colunas como str


hist = pd.read_csv(arq_vol, dtype=col_types,
                        storage_options = {'linked_service' : linked_service_raw})



hist.drop(['Unnamed: 0','index'],axis=1,inplace=True)                       # var não necessária
hist['VEDI_VOL'] = hist['VEDI_VOL'].astype(float)
hist['VEDI_DAT_VEN'] = pd.to_datetime(hist['VEDI_DAT_VEN'])

#organizando o Coreto!
hist = hist[['CLIE_NUM_CPF_CNPJ', 'CLIE_NOM_CLIENTE','VEDI_COD_CLI', 'VEDI_VOL', 'VEDI_DAT_VEN', 'ProdutoD', 'AECO']]

hist.rename(columns={'CLIE_NUM_CPF_CNPJ': 'CNPJ'}, errors='raise', inplace=True)

hist = hist[hist['AECO']=='USINA'].reset_index(drop=True)
hist['CNPJ'] = hist['CNPJ'].astype(str).apply(lambda x: x.zfill(14))    #Transforma CPNJ em string
hist = hist[~hist['CNPJ'].str.startswith('000')]                        # removendo possiveis CPFs

hist = hist[hist['VEDI_VOL'] > 0].sort_values('VEDI_DAT_VEN')                  # Eliminando Vendas negativas ou zeradas
hist_v = hist.copy()

### Histórico Processado na Onda2

In [4]:
#HIST PROCESSADO NO INICIO DA ONDA 2

#del(col_names, col_types)                                                  # eliminando dados de outra leitura
col_names = pd.read_csv(arq_usi, nrows=0).columns                           # coletando informações de cabeçalho
col_types = {col: str for col in col_names if col not in {}}                # todas as colunas como str

hist = pd.read_csv(arq_usi, dtype=col_types,
                        storage_options = {'linked_service' : linked_service_raw})

hist.drop('keymodel',axis=1,inplace=True)                                   # var não necessária
hist['VEDI_VOL'] = hist['VEDI_VOL'].astype(float)
hist['VEDI_DAT_VEN'] = pd.to_datetime(hist['VEDI_DAT_VEN'])

#organizando o Coreto!
hist = hist[['CLIE_NUM_CPF_CNPJ', 'CLIE_NOM_CLIENTE','VEDI_COD_CLI', 'VEDI_VOL', 'VEDI_DAT_VEN', 'ProdutoD', 'AECO']]

hist.rename(columns={'CLIE_NUM_CPF_CNPJ': 'CNPJ'}, errors='raise', inplace=True)

hist = hist[hist['AECO']=='USINA'].sort_values('VEDI_DAT_VEN').reset_index(drop=True)


## Comparação

In [5]:
hist_v.head(10)

In [6]:
hist.head(10)

In [7]:
hist_v[(hist_v['CLIE_NOM_CLIENTE']=='OUROESTE BIOENERGIA LTDA') & (hist_v['VEDI_DAT_VEN']<'2017-01-01')]

In [8]:
hist[(hist['CLIE_NOM_CLIENTE']=='OUROESTE BIOENERGIA LTDA')&(hist['VEDI_DAT_VEN']<'2017-01-01')]

### Percentual de USINAS que a Vibra realizou vendas no ano


In [9]:
hist_v['ANO'] = hist_v['VEDI_DAT_VEN'].dt.strftime('%Y') 
hist_v['ANOMES'] = hist_v['VEDI_DAT_VEN'].dt.strftime('%Y-%m')

hist['ANO'] = hist['VEDI_DAT_VEN'].dt.strftime('%Y')

In [10]:
# agrupando as vendas por cnpj e ano trazendo volume médio, numero de vendas e volume total de DIESEL

df_vg1 = hist_v.groupby(
                        ['CNPJ','CLIE_NOM_CLIENTE','ANO'])['VEDI_VOL'].agg(
                            Vol_Total = 'sum', Freq='count', Vol_Medio='mean').reset_index()

df_vg = hist_v.groupby(
                        ['CNPJ','CLIE_NOM_CLIENTE','ANOMES'])['VEDI_VOL'].agg(
                            Vol_Total = 'sum', Freq='count', Vol_Medio='mean').reset_index()

df_g = hist.groupby(
                    ['CNPJ','CLIE_NOM_CLIENTE','ANO'])['VEDI_VOL'].agg(
                        Vol_Total = 'sum', Freq='count', Vol_Medio='mean').reset_index()

In [11]:
def participacao(df, freq):
    temp = sorted(df[freq].value_counts().index.tolist())
    print('Percentual de USINAS que a Vibra realizou vendas no', freq)
    for i in temp[:-1]:                                                             # até o ultimo ano completo
        df_a = df[df[freq]==i][['CNPJ',freq, 'Vol_Total','Freq','Vol_Medio']]
        us = usinas.merge(df_a, how='left', on='CNPJ')
        us[freq]=i
        print('{}: {:2.2%}'.format(i, us[~us['Freq'].isnull()].shape[0]/len(us)))

In [12]:
participacao(df_vg, 'ANOMES')